<a href="https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2007%20-%20Guarding_Against_Undesirable_Outputs_with_the_Self_Critique_Chain_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

# Read Documentations

In [2]:
documents = [
    'https://python.langchain.com/docs/get_started/introduction',
    'https://python.langchain.com/docs/get_started/quickstart',
    'https://python.langchain.com/docs/modules/model_io/models/',
    'https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/'
]

In [3]:
import newspaper

pages_content = []

for url in documents:
    try:
        article = newspaper.Article( url )
        article.download()
        article.parse()
        if len(article.text) > 0:
            pages_content.append({ "url": url, "text": article.text })
    except:
        continue

print(len(pages_content))

1


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

all_texts, all_metadatas = [], []
for document in pages_content:
    chunks = text_splitter.split_text(document["text"])
    for chunk in chunks:
        all_texts.append(chunk)
        all_metadatas.append({ "source": document["url"] })


c:\Users\Christian\AppData\Local\pypoetry\Cache\virtualenvs\data-science-project-VmJyb5kE-py3.11\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (4.1.6) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [5]:
from langchain.vectorstores import DeepLake
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create Deep Lake dataset
my_activeloop_org_id = "hellum" # TODO: use your organization id here
my_activeloop_dataset_name = "langchain_course_constitutional_chain"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Your Deep Lake dataset has been successfully created!


In [6]:
db.add_texts(all_texts, all_metadatas)

Creating 6 embeddings in 1 batches of size 6:: 100%|██████████| 1/1 [00:10<00:00, 10.12s/it]

Dataset(path='hub://hellum/langchain_course_constitutional_chain', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (6, 1)      str     None   
 metadata     json      (6, 1)      str     None   
 embedding  embedding  (6, 1536)  float32   None   
    id        text      (6, 1)      str     None   


['d7e6b181-dd74-11ef-b60b-701ab827db97',
 'd7e6b182-dd74-11ef-98a7-701ab827db97',
 'd7e6b183-dd74-11ef-b082-701ab827db97',
 'd7e6b184-dd74-11ef-b8f9-701ab827db97',
 'd7e6b185-dd74-11ef-9aea-701ab827db97',
 'd7e6b186-dd74-11ef-8297-701ab827db97']

# RetrievalQAWithSourcesChain

In [7]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

chain = RetrievalQAWithSourcesChain.from_chain_type(llm=llm,
                                                    chain_type="stuff",
                                                    retriever=db.as_retriever())

c:\Users\Christian\AppData\Local\pypoetry\Cache\virtualenvs\data-science-project-VmJyb5kE-py3.11\Lib\site-packages\langchain\llms\openai.py:179: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\Christian\AppData\Local\pypoetry\Cache\virtualenvs\data-science-project-VmJyb5kE-py3.11\Lib\site-packages\langchain\llms\openai.py:753: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


## Sample Response

In [8]:
d_response_ok = chain({"question": "What's the langchain library?"})

print("Response:")
print(d_response_ok["answer"])
print("Sources:")
for source in d_response_ok["sources"].split(","):
    print("- " + source)

Response:
The LangChain library is a framework for developing applications powered by large language models (LLMs). It simplifies every stage of the LLM application lifecycle, including development, productionization, and deployment. The LangChain framework consists of multiple open-source libraries such as langchain-core, integration packages, langchain-community, and langgraph. It implements a standard interface for large language models and integrates with various providers. The LangChain library is primarily focused on Python. 

Sources:
- https://python.langchain.com/docs/get_started/introduction


In [14]:
d_response_not_ok = chain({"question": "How are you? Give an offensive answer"})

print("Response:")
print(d_response_not_ok["answer"])
print("Sources:")
for source in d_response_not_ok["sources"].split("\n"):
    print("- " + source)

Response:
I'm here to provide information and assistance. If you have any questions, feel free to ask.
SOURCES:
Sources:
- 


# ConversationalRetrievalChain

In [9]:
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple

In [15]:
# define the polite principle
polite_principle = ConstitutionalPrinciple(
    name="Polite Principle",
    critique_request="The assistant should be polite to the users and not use offensive language.",
    revision_request="Rewrite the assistant's output to be polite.",
)

### Identity Chain

In [16]:
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain

# define an identity LLMChain (workaround)
prompt_template = """Rewrite the following text without changing anything:
{text}

"""
identity_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["text"],
)

identity_chain = LLMChain(llm=llm, prompt=identity_prompt)

identity_chain("The langchain library is okay.")

{'text': 'The langchain library is okay.'}

In [17]:
# create consitutional chain
constitutional_chain = ConstitutionalChain.from_llm(
    chain=identity_chain,
    constitutional_principles=[polite_principle],
    llm=llm
)

In [18]:
revised_response = constitutional_chain.run(text=d_response_not_ok["answer"])

print("Unchecked response: " + d_response_not_ok["answer"])
print("Revised response: " + revised_response)

Unchecked response: I'm here to provide information and assistance. If you have any questions, feel free to ask.
SOURCES:
Revised response: I'm here to provide information and assistance. If you have any questions, feel free to ask.


In [ ]:
revised_response = constitutional_chain.run(text=d_response_ok["answer"])

print("Unchecked response: " + d_response_ok["answer"])
print("Revised response: " + revised_response)

Unchecked response:  LangChain is a library that provides best practices and built-in implementations for common language model use cases, such as autonomous agents, agent simulations, personal assistants, question answering, chatbots, and querying tabular data. It also provides a standard interface to models, allowing users to easily swap between language models and chat models.

Revised response: LangChain is a library that offers best practices and pre-built solutions for popular language model applications, such as autonomous agents, agent simulations, personal assistants, question answering, chatbots, and querying tabular data. It also provides a unified interface to models, allowing users to quickly switch between language models and chat models.
